In [25]:
import tensorflow as tf
import numpy as np

Vi starter med at loaded den givne data ind. Her er dataen sepereret i lyd med musik og lyd uden musik. Det skal vi så sætte sammen i et array og lave en array som indeholder om hvilken type hver styk data er. Her kan vi bruge numpy siden hver fil er i .npy format

In [26]:
music_data = np.load("music_data.npy")
other_data = np.load("other_data.npy")
unknown_data = np.load("test_data.npy")

Data layoutet er her $10500$ 2d arrays af $30\times79$ som repræsentere et log-mel spektogram udregnet fra den givne lyd sample. $x$ dimensionen er frekvens båndene mens $y$ er tids rammen

Nu skal vi sætte `music_data` og `other_data` sammen til et array og lave en array som indeholder hvilken type hver lyd sample er. Vi vil også gerne have noget test data så vi kan evaluere hvor godt netværket er uden at overfitte Det gør vi ved at efterlade 1000 fra `music_dat` og `other_data`

In [27]:
music_train_data = music_data[0: -1000]
other_train_data = other_data[0: -1000]
train_data = (
	np.concatenate((music_train_data, other_train_data)),
	np.concatenate((
		np.full((len(music_train_data)), (1)), 
		np.full((len(other_train_data)), (0))
	))
)	

music_test_data = music_data[len(music_data) - 1000:None]
other_test_data = other_data[len(other_data) - 1000:None]
test_data = (
	np.concatenate((music_test_data, other_test_data)),
	np.concatenate((
		np.full((len(music_test_data)), (1)),
		np.full((len(other_test_data)), (0))
	))
)


Nu har vi lavet dataset til test og træning, nu skal vi sætte vores model op

In [28]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Reshape((30, 79, 1), input_shape = (30, 79)),
	tf.keras.layers.Conv2D(3, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(1024, activation = "sigmoid"),
	tf.keras.layers.Dense(512, activation = "sigmoid"),
	tf.keras.layers.Dense(1, activation = "sigmoid")
])

model.build(input_shape=(30, 79))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 30, 79, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 77, 3)         30        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 38, 3)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 36, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 18, 32)        0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 16, 64)        

Nu hvor vi har en model, så skal den bare trænes

In [29]:
predictions = model(train_data[0][:10]).numpy()
predictions

array([[0.3153832 ],
       [0.3103457 ],
       [0.3139407 ],
       [0.31499943],
       [0.31837097],
       [0.31653255],
       [0.31139585],
       [0.31477883],
       [0.3125565 ],
       [0.31083167]], dtype=float32)

Nu laver vi en loss funktion, siden vi kun har et output så bruger vi binarycrossentropy

In [30]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Så træner vi modellen med dataen over 10 epochs

In [31]:
history = model.fit(train_data[0], train_data[1], epochs=10)

Epoch 1/10
594/594 [==============================] - 14s 23ms/step - loss: 0.2724 - accuracy: 0.8797
Epoch 2/10
594/594 [==============================] - 14s 23ms/step - loss: 0.1187 - accuracy: 0.9587
Epoch 3/10
594/594 [==============================] - 15s 25ms/step - loss: 0.0837 - accuracy: 0.9697
Epoch 4/10
594/594 [==============================] - 14s 23ms/step - loss: 0.0615 - accuracy: 0.9780
Epoch 5/10
594/594 [==============================] - 15s 25ms/step - loss: 0.0426 - accuracy: 0.9856
Epoch 6/10
594/594 [==============================] - 15s 25ms/step - loss: 0.0347 - accuracy: 0.9878
Epoch 7/10
594/594 [==============================] - 13s 23ms/step - loss: 0.0284 - accuracy: 0.9911
Epoch 8/10
594/594 [==============================] - 13s 22ms/step - loss: 0.0267 - accuracy: 0.9907
Epoch 9/10
594/594 [==============================] - 14s 23ms/step - loss: 0.0178 - accuracy: 0.9951
Epoch 10/10
594/594 [==============================] - 13s 23ms/step - loss: 0.017

In [32]:
train_evaluation = model.evaluate(train_data[0], train_data[1])
test_evaluation = model.evaluate(test_data[0], test_data[1])
print("Training set accuracy %.3f%% and loss: %.3f%%" % 
	(train_evaluation[1] * 100, train_evaluation[0] * 100))
print("Test set accuracy %.3f%% and loss: %.3f%%" % 
	(test_evaluation[1] * 100, test_evaluation[0] * 100))

63/63 [==============================] - 1s 9ms/step - loss: 0.0725 - accuracy: 0.9765
Training set accuracy 99.842% and loss: 0.677%
Test set accuracy 97.650% and loss: 7.250%


In [40]:
data = model(unknown_data).numpy()

In [44]:
f = open("predictions.txt", "w")

first = True

for prediction in data:
	if first:
		first = False
	else:
		f.write("\n")
	if prediction > 0.5:
		f.write("1")
	else:
		f.write("0")
f.close()